In [ ]:
# -*- coding: utf-8 -*-
"""
R&R(Reel & Read) 프로젝트를 위한 추천 알고리즘 개발 스크립트입니다.

[작업 순서]
1. 전처리된 데이터 로드
2. 콘텐츠 기반 필터링(CBF) 모델 생성 (작품 유사도)
3. 사용자 기반 협업 필터링(User-CF) 모델 생성 (사용자 유사도)
4. 추천 데이터 생성
   - Hybrid 방식의 작품 추천 (Recommend_work)
   - 사용자 추천 (Recommend_user)
   - 사용자 관심 장르 추출 (User_interest)
5. 최종 결과를 DB Seeding용 CSV 파일로 저장
"""

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import numpy as np
import ast # 문자열로 된 리스트를 실제 리스트로 변환하기 위해 사용
from tqdm import tqdm

# ==============================================================================
# Step 1: 전처리된 데이터 로드
# ==============================================================================
print("Step 1: 전처리된 데이터를 로드합니다.")
works_df = pd.read_csv("works_processed.csv")
ratings_df = pd.read_csv("ratings_processed.csv")

# genres 컬럼이 문자열화된 리스트이므로, ast.literal_eval을 사용해 다시 리스트로 변환
works_df['genres'] = works_df['genres'].apply(ast.literal_eval)

print("데이터 로드 완료.")


# ==============================================================================
# Step 2: 콘텐츠 기반 필터링(CBF) 모델 생성
# ==============================================================================
print("\nStep 2: 콘텐츠 기반 필터링(CBF) 모델을 생성합니다 (작품 유사도).")

# TF-IDF 벡터화를 위해 장르 리스트를 공백으로 구분된 문자열로 변환
works_df['genres_str'] = works_df['genres'].apply(lambda x: ' '.join(x))

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(works_df['genres_str'])

# 작품 간 코사인 유사도 계산
work_similarity = cosine_similarity(tfidf_matrix, tfidf_matrix)

# 나중에 쉽게 조회할 수 있도록 work_id를 인덱스로 하는 DataFrame으로 변환
work_sim_df = pd.DataFrame(work_similarity, index=works_df['work_id'], columns=works_df['work_id'])

print("작품 유사도 행렬 생성 완료. Shape:", work_sim_df.shape)


# ==============================================================================
# Step 3: 사용자 기반 협업 필터링(User-CF) 모델 생성
# ==============================================================================
print("\nStep 3: 사용자 기반 협업 필터링(User-CF) 모델을 생성합니다 (사용자 유사도).")

# 사용자-아이템 행렬(피벗 테이블) 생성
user_item_matrix = ratings_df.pivot_table(index='userId', columns='work_id', values='rating').fillna(0)

# 사용자 간 코사인 유사도 계산
user_similarity = cosine_similarity(user_item_matrix)

# 사용자 ID를 인덱스로 하는 DataFrame으로 변환
user_sim_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

print("사용자 유사도 행렬 생성 완료. Shape:", user_sim_df.shape)


# ==============================================================================
# Step 4: 추천 데이터 생성
# ==============================================================================
print("\nStep 4: DB에 저장할 추천 데이터를 생성합니다.")

# --- 4.1 `db_recommend_work.csv` 생성 (하이브리드 작품 추천) ---
# 모든 사용자에 대해 추천 목록을 미리 생성 (오프라인 방식)
all_recommendations = []
top_n_similar_users = 10 # 각 사용자에 대해 참고할 유사 사용자 수
top_k_works = 20 # 각 사용자에게 최종 추천할 작품 수
high_rating_threshold = 4.0 # 사용자가 '좋아한다'고 판단하는 평점 기준

# tqdm을 사용하여 진행 상황 표시
for user_id in tqdm(user_sim_df.index, desc="작품 추천 목록 생성 중"):
    # (1) CF 기반 추천: 나와 비슷한 사용자들이 좋아한 작품
    similar_users = user_sim_df[user_id].sort_values(ascending=False)[1:top_n_similar_users+1].index
    similar_users_ratings = ratings_df[ratings_df['userId'].isin(similar_users)]
    recommended_works_cf = similar_users_ratings[similar_users_ratings['rating'] >= high_rating_threshold]

    # (2) 사용자가 이미 본 작품은 제외
    seen_works = ratings_df[ratings_df['userId'] == user_id]['work_id'].tolist()
    recommended_works_cf = recommended_works_cf[~recommended_works_cf['work_id'].isin(seen_works)]

    if not recommended_works_cf.empty:
        # 유사 사용자의 평점을 가중치로 사용하여 작품 점수 계산
        work_scores = recommended_works_cf.groupby('work_id')['rating'].mean()
        # 점수가 높은 순으로 정렬
        final_recs = work_scores.sort_values(ascending=False).head(top_k_works).reset_index()

        for _, row in final_recs.iterrows():
            all_recommendations.append({'user_id': user_id, 'work_id': int(row['work_id']), 'score': row['rating']})

db_recommend_work_df = pd.DataFrame(all_recommendations)
print("`Recommend_work` 데이터 생성 완료.")


# --- 4.2 `db_recommend_user.csv` 생성 (사용자 추천) ---
all_user_recs = []
top_n_users_to_recommend = 10 # 추천할 사용자 수

for user_id in tqdm(user_sim_df.index, desc="사용자 추천 목록 생성 중"):
    similar_users = user_sim_df[user_id].sort_values(ascending=False)[1:top_n_users_to_recommend+1]
    for target_id, score in similar_users.items():
        all_user_recs.append({'user_id': user_id, 'target_id': target_id, 'score': score})

db_recommend_user_df = pd.DataFrame(all_user_recs)
print("`Recommend_user` 데이터 생성 완료.")


# --- 4.3 `db_user_interest.csv` 생성 (사용자 관심 장르 추출) ---
# 장르 이름과 ID를 매핑하는 딕셔너리 생성
db_genres_df = pd.read_csv("db_genres.csv") # 이전 단계에서 생성한 파일
genre_to_id = dict(zip(db_genres_df['name'], db_genres_df['genre_id']))

user_interests = []
top_n_genres = 3 # 각 사용자의 대표 장르 수

# 사용자가 높게 평가한 작품들의 장르를 분석
high_rated_works = ratings_df[ratings_df['rating'] >= high_rating_threshold]
user_genre_data = pd.merge(high_rated_works, works_df, on='work_id')

for user_id in tqdm(user_genre_data['userId'].unique(), desc="사용자 관심 장르 추출 중"):
    user_genres = user_genre_data[user_genre_data['userId'] == user_id]['genres'].explode()
    top_genres = user_genres.value_counts().nlargest(top_n_genres).index.tolist()

    for genre_name in top_genres:
        genre_id = genre_to_id.get(genre_name)
        if genre_id:
            user_interests.append({'user_id': user_id, 'genre_id': genre_id})

db_user_interest_df = pd.DataFrame(user_interests)
print("`User_interest` 데이터 생성 완료.")


# ==============================================================================
# Step 5: 최종 결과를 DB Seeding용 CSV 파일로 저장
# ==============================================================================
print("\nStep 5: 생성된 추천 데이터를 CSV 파일로 저장합니다.")

db_recommend_work_df.to_csv("db_recommend_work.csv", index=False, encoding='utf-8-sig')
db_recommend_user_df.to_csv("db_recommend_user.csv", index=False, encoding='utf-8-sig')
db_user_interest_df.to_csv("db_user_interest.csv", index=False, encoding='utf-8-sig')

print("\n모든 추천 알고리즘 실행 및 DB 파일 생성이 완료되었습니다.")
print("- db_recommend_work.csv: 각 사용자별 추천 작품 목록")
print("- db_recommend_user.csv: 각 사용자별 추천 사용자 목록")
print("- db_user_interest.csv: 각 사용자별 관심 장르 목록")

Step 1: 전처리된 데이터를 로드합니다.
데이터 로드 완료.

Step 2: 콘텐츠 기반 필터링(CBF) 모델을 생성합니다 (작품 유사도).
작품 유사도 행렬 생성 완료. Shape: (29800, 29800)

Step 3: 사용자 기반 협업 필터링(User-CF) 모델을 생성합니다 (사용자 유사도).
사용자 유사도 행렬 생성 완료. Shape: (610, 610)

Step 4: DB에 저장할 추천 데이터를 생성합니다.


작품 추천 목록 생성 중: 100%|██████████| 610/610 [00:03<00:00, 169.44it/s]


`Recommend_work` 데이터 생성 완료.


사용자 추천 목록 생성 중: 100%|██████████| 610/610 [00:00<00:00, 4385.16it/s]


`Recommend_user` 데이터 생성 완료.


사용자 관심 장르 추출 중: 100%|██████████| 609/609 [00:01<00:00, 506.59it/s]

`User_interest` 데이터 생성 완료.

Step 5: 생성된 추천 데이터를 CSV 파일로 저장합니다.

모든 추천 알고리즘 실행 및 DB 파일 생성이 완료되었습니다.
- db_recommend_work.csv: 각 사용자별 추천 작품 목록
- db_recommend_user.csv: 각 사용자별 추천 사용자 목록
- db_user_interest.csv: 각 사용자별 관심 장르 목록


In [ ]:
# -*- coding: utf-8 -*-
"""
R&R(Reel & Read) 프로젝트를 위한 추천 알고리즘 개발 스크립트입니다.

[작업 순서]
1. 전처리된 데이터 로드
2. 콘텐츠 기반 필터링(CBF) 모델 생성 (작품 유사도)
3. 사용자 기반 협업 필터링(User-CF) 모델 생성 (사용자 유사도)
4. 추천 데이터 생성
   - Hybrid 방식의 작품 추천 (Recommend_work)
   - 사용자 추천 (Recommend_user)
   - 사용자 관심 장르 추출 (User_interest)
5. 최종 결과를 DB Seeding용 CSV 파일로 저장
"""

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import numpy as np
import ast # 문자열로 된 리스트를 실제 리스트로 변환하기 위해 사용
from tqdm import tqdm

# ==============================================================================
# Step 1: 전처리된 데이터 로드
# ==============================================================================
print("Step 1: 전처리된 데이터를 로드합니다.")
works_df = pd.read_csv("works_.csv")
ratings_df = pd.read_csv("ratings_.csv")
posts_df = pd.read_csv("posts.csv")

# genres 컬럼이 문자열화된 리스트이므로, ast.literal_eval을 사용해 다시 리스트로 변환
works_df['genres'] = works_df['genres'].apply(ast.literal_eval)

print("데이터 로드 완료.")


# ==============================================================================
# Step 2: 콘텐츠 기반 필터링(CBF) 모델 생성
# ==============================================================================
print("\nStep 2: 콘텐츠 기반 필터링(CBF) 모델을 생성합니다 (작품 유사도).")

# TF-IDF 벡터화를 위해 장르 리스트를 공백으로 구분된 문자열로 변환
works_df['genres_str'] = works_df['genres'].apply(lambda x: ' '.join(x))

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(works_df['genres_str'])

# 작품 간 코사인 유사도 계산
work_similarity = cosine_similarity(tfidf_matrix, tfidf_matrix)

# 나중에 쉽게 조회할 수 있도록 work_id를 인덱스로 하는 DataFrame으로 변환
work_sim_df = pd.DataFrame(work_similarity, index=works_df['work_id'], columns=works_df['work_id'])

print("작품 유사도 행렬 생성 완료. Shape:", work_sim_df.shape)


# ==============================================================================
# Step 3: 사용자 기반 협업 필터링(User-CF) 모델 생성
# ==============================================================================
print("\nStep 3: 사용자 기반 협업 필터링(User-CF) 모델을 생성합니다 (사용자 유사도).")

# 사용자-아이템 행렬(피벗 테이블) 생성
user_item_matrix = ratings_df.pivot_table(index='userId', columns='work_id', values='rating').fillna(0)

# 사용자 간 코사인 유사도 계산
user_similarity = cosine_similarity(user_item_matrix)

# 사용자 ID를 인덱스로 하는 DataFrame으로 변환
user_sim_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

print("사용자 유사도 행렬 생성 완료. Shape:", user_sim_df.shape)


# ==============================================================================
# Step 4: 추천 데이터 생성
# ==============================================================================
print("\nStep 4: DB에 저장할 추천 데이터를 생성합니다.")

# --- 4.1 `db_recommend_work.csv` 생성 (하이브리드 작품 추천) ---
# 모든 사용자에 대해 추천 목록을 미리 생성 (오프라인 방식)
all_recommendations = []
top_n_similar_users = 10 # 각 사용자에 대해 참고할 유사 사용자 수
top_k_works = 20 # 각 사용자에게 최종 추천할 작품 수
high_rating_threshold = 4.0 # 사용자가 '좋아한다'고 판단하는 평점 기준

# tqdm을 사용하여 진행 상황 표시
for user_id in tqdm(user_sim_df.index, desc="작품 추천 목록 생성 중"):
    # (1) CF 기반 추천: 나와 비슷한 사용자들이 좋아한 작품
    similar_users = user_sim_df[user_id].sort_values(ascending=False)[1:top_n_similar_users+1].index
    similar_users_ratings = ratings_df[ratings_df['userId'].isin(similar_users)]
    recommended_works_cf = similar_users_ratings[similar_users_ratings['rating'] >= high_rating_threshold]

    # (2) 사용자가 이미 본 작품은 제외
    seen_works = ratings_df[ratings_df['userId'] == user_id]['work_id'].tolist()
    recommended_works_cf = recommended_works_cf[~recommended_works_cf['work_id'].isin(seen_works)]

    if not recommended_works_cf.empty:
        # 유사 사용자의 평점을 가중치로 사용하여 작품 점수 계산
        work_scores = recommended_works_cf.groupby('work_id')['rating'].mean()
        # 점수가 높은 순으로 정렬
        final_recs = work_scores.sort_values(ascending=False).head(top_k_works).reset_index()

        for _, row in final_recs.iterrows():
            all_recommendations.append({'user_id': user_id, 'work_id': int(row['work_id']), 'score': row['rating']})

db_recommend_work_df = pd.DataFrame(all_recommendations)
print("`Recommend_work` 데이터 생성 완료.")


# --- 4.2 `db_recommend_user.csv` 생성 (사용자 추천) ---
all_user_recs = []
top_n_users_to_recommend = 10 # 추천할 사용자 수

for user_id in tqdm(user_sim_df.index, desc="사용자 추천 목록 생성 중"):
    similar_users = user_sim_df[user_id].sort_values(ascending=False)[1:top_n_users_to_recommend+1]
    for target_id, score in similar_users.items():
        all_user_recs.append({'user_id': user_id, 'target_id': target_id, 'score': score})

db_recommend_user_df = pd.DataFrame(all_user_recs)
print("`Recommend_user` 데이터 생성 완료.")


# --- 4.3 `db_user_interest.csv` 생성 (사용자 관심 장르 추출) ---
# 장르 이름과 ID를 매핑하는 딕셔너리 생성
db_genres_df = pd.read_csv("db_genres.csv") # 이전 단계에서 생성한 파일
genre_to_id = dict(zip(db_genres_df['name'], db_genres_df['genre_id']))

user_interests = []
top_n_genres = 3 # 각 사용자의 대표 장르 수

# 사용자가 높게 평가한 작품들의 장르를 분석
# (데이터 로드 부분에 posts 테이블을 읽어오는 코드 추가)
posts_df = pd.read_sql("SELECT user_id, work_id, ai_emotion FROM posts WHERE post_type = 'review'", db.bind)

# 긍정적인(positive) 리뷰를 남긴 작품만 필터링
positive_review_works = posts_df[posts_df['ai_emotion'] == 'positive']

# 긍정 리뷰가 달린 작품들의 장르를 분석
# user_id 컬럼명이 ratings_df와 다르다면 맞춰줘야 합니다. (userId -> user_id)
user_genre_data = pd.merge(positive_review_works, works_df, on='work_id')

for user_id in tqdm(user_genre_data['userId'].unique(), desc="사용자 관심 장르 추출 중"):
    user_genres = user_genre_data[user_genre_data['userId'] == user_id]['genres'].explode()
    top_genres = user_genres.value_counts().nlargest(top_n_genres).index.tolist()

    for genre_name in top_genres:
        genre_id = genre_to_id.get(genre_name)
        if genre_id:
            user_interests.append({'user_id': user_id, 'genre_id': genre_id})

db_user_interest_df = pd.DataFrame(user_interests)
print("`User_interest` 데이터 생성 완료.")


# ==============================================================================
# Step 5: 최종 결과를 DB Seeding용 CSV 파일로 저장
# ==============================================================================
print("\nStep 5: 생성된 추천 데이터를 CSV 파일로 저장합니다.")

db_recommend_work_df.to_csv("db_recommend_work.csv", index=False, encoding='utf-8-sig')
db_recommend_user_df.to_csv("db_recommend_user.csv", index=False, encoding='utf-8-sig')
db_user_interest_df.to_csv("db_user_interest.csv", index=False, encoding='utf-8-sig')

print("\n모든 추천 알고리즘 실행 및 DB 파일 생성이 완료되었습니다.")
print("- db_recommend_work.csv: 각 사용자별 추천 작품 목록")
print("- db_recommend_user.csv: 각 사용자별 추천 사용자 목록")
print("- db_user_interest.csv: 각 사용자별 관심 장르 목록")